# Innomatics Research Labs – Technical Evaluation

**Name:** Ponna Pravalika
**Date:** 31 January 2026


In [2]:
import pandas as pd
import numpy as np
import sqlite3


In [6]:
# Load orders CSV file
orders = pd.read_csv("orders.csv")
#Preview
orders.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [7]:
# Load users JSON file
users = pd.read_json("users.json")
#Preview
users.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [5]:
# Create database connection
conn = sqlite3.connect("restaurants.db")

# Read SQL file
with open("restaurants.sql", "r") as file:
    sql_script = file.read()

# Execute SQL script
conn.executescript(sql_script)

# Load restaurants table into dataframe
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [9]:
# Merge Data Orders+Users
orders_users = pd.merge(
    orders,
    users,
    on="user_id",
    how="left"
)


In [10]:
# Add Restaurants
final_df = pd.merge(
    orders_users,
    restaurants,
    on="restaurant_id",
    how="left"
)


In [11]:
# Saving Final Dataset
final_df.to_csv("final_food_delivery_dataset.csv", index=False)


## MCQ 1
**Question:** Which city has the highest total revenue  (total_amount) from Gold members?


In [13]:
# Filter only Gold members and calculate total revenue by city
gold_revenue = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
)

gold_revenue.sort_values(ascending=False)


,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


**Answer:** The city with the highest total revenue from Gold members is **"Chennai"**.


## MCQ 2
**Question:** Which cuisine has the highest average order value across all orders?


In [14]:
# Calculate average order value for each cuisine
final_df.groupby("cuisine")["total_amount"].mean().sort_values(ascending=False)


,total_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


**Answer:** The cuisine with the highest average order value is **"Mexican"**.


## MCQ 3
**Question:** How many distinct users placed orders worth more than ₹1000 in total (sum of all their orders)?


In [15]:
# Calculate total spend per user
user_spend = final_df.groupby("user_id")["total_amount"].sum()

# Count users spending more than 1000
count_users = (user_spend > 1000).sum()
count_users


np.int64(2544)

**Answer:** The number of users who spent more than ₹1000 falls in the range **">2000"**.


## MCQ 4
**Question:** Which restaurant rating range generated the highest total revenue?


In [37]:
# Create rating ranges
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ["3.0–3.5", "3.6–4.0", "4.1–4.5", "4.6–5.0"]

final_df["rating_range"] = pd.cut(final_df["rating"], bins=bins, labels=labels)

# Calculate revenue by rating range
final_df.groupby("rating_range", observed=True)["total_amount"].sum()



,total_amount
rating_range,
3.0–3.5,1881754.57
3.6–4.0,1717494.41
4.1–4.5,1960326.26
4.6–5.0,2197030.75


**Answer:** The rating range with the highest total revenue is **"4.6-5.0"**.


## MCQ 5
**Question:** Among Gold members, which city has the highest average order value?


In [17]:
# Filter Gold members and calculate average order value by city
gold_city_avg = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .mean()
    .sort_values(ascending=False)
)

gold_city_avg


,total_amount
city,
Chennai,808.459080
Hyderabad,806.421034
Bangalore,793.223756
Pune,781.162243


**Answer:** Among Gold members, the city with the highest average order value is **"Chennai"**.


## MCQ 6
**Question:** Which cuisine has the lowest number of distinct restaurants but still contributes significant revenue?


In [18]:
# Count distinct restaurants per cuisine
restaurant_count = final_df.groupby("cuisine")["restaurant_id"].nunique()

# Total revenue per cuisine
revenue_by_cuisine = final_df.groupby("cuisine")["total_amount"].sum()

restaurant_count, revenue_by_cuisine


(cuisine
 Chinese    120
 Indian     126
 Italian    126
 Mexican    128
 Name: restaurant_id, dtype: int64,
 cuisine
 Chinese    1930504.65
 Indian     1971412.58
 Italian    2024203.80
 Mexican    2085503.09
 Name: total_amount, dtype: float64)

**Answer:** The cuisine with fewer restaurants but significant revenue contribution is **"Chinese"**.


## MCQ 7
**Question:** What percentage of total orders were placed by Gold members?  (Rounded to nearest integer)


In [28]:
# Percentage of total orders placed by Gold memebers
gold_orders = final_df[final_df["membership"] == "Gold"].shape[0]
total_orders = final_df.shape[0]

percentage = round((gold_orders / total_orders) * 100)
percentage


50

**Answer:**  **"50%"** of total orders were placed by Gold members.


## MCQ 8
**Question:** Which restaurant has the highest average order value but less than 20 total orders?


In [24]:
# Calculate order count and average order value per restaurant
restaurant_stats = (
    final_df.groupby("restaurant_name_x")
    .agg(
        order_count=("order_id", "count"),
        avg_order_value=("total_amount", "mean")
    )
)

# Filter restaurants with less than 20 orders
filtered_restaurants = restaurant_stats[restaurant_stats["order_count"] < 20]

# Sort by highest average order value
filtered_restaurants.sort_values("avg_order_value", ascending=False)


,order_count,avg_order_value
restaurant_name_x,,
Hotel Dhaba Multicuisine,13,1040.222308
Sri Mess Punjabi,12,1029.180833
Ruchi Biryani Punjabi,16,1002.140625
Sri Delights Pure Veg,18,989.467222
Classic Kitchen Family Restaurant,19,973.167895
...,...,...
Annapurna Tiffins Punjabi,19,621.828947
Darbar Tiffins Non-Veg,18,596.815556
Darbar Restaurant Punjabi,14,589.972857


**Answer:** The restaurant with the highest average order value and fewer than 20 orders is **"Ruchi Foods Chinese"**.


## MCQ 9
**Question:** Which combination contributes the highest revenue?


In [26]:
# Revenue by membership and cuisine
combo_revenue = (
    final_df.groupby(["membership", "cuisine"])["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

combo_revenue


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

**Answer:** The combination contributing the highest revenue is **"Gold + Italian"**.


## MCQ 10
**Question:** During which quarter of the year is the total revenue highest?


In [38]:
final_df["order_date"] = pd.to_datetime(final_df["order_date"], dayfirst=True)
final_df["quarter"] = final_df["order_date"].dt.quarter

quarter_revenue = final_df.groupby("quarter")["total_amount"].sum()
quarter_revenue


,total_amount
quarter,
1,2010626.64
2,1945348.72
3,2037385.10
4,2018263.66


**Answer:** The quarter with the highest total revenue is **"Q3 (Jul–Sep)"**.


## Numerical type Question 1
**Question:** How many total orders were placed by users with Gold membership?

In [29]:
# Total orders placed by Gold members
gold_orders = final_df[final_df["membership"] == "Gold"].shape[0]
gold_orders


4987

**Answer:** Users with Gold membership placed a total of **"4987"** orders.


## Numerical type Question 2
**Question:** What is the total revenue (rounded to the nearest integer) generated from orders placed in Hyderabad city?


In [30]:
# Filter orders placed in Hyderabad and calculate total revenue
hyderabad_revenue = final_df[final_df["city"] == "Hyderabad"]["total_amount"].sum()

hyderabad_revenue


np.float64(1889366.58)

**Answer:** The total revenue generated from orders placed in Hyderabad city is approximately **"1889367"**.


## Numerical type Question 3
**Question:** How many distinct users placed at least one order?



In [31]:
# Counting distinct users
final_df["user_id"].nunique()
distinct_users = final_df["user_id"].nunique()
distinct_users


2883

**Answer:** A total of **"2883" distinct users** placed at least one order.


## Numerical type Question 4
**Question:** What is the average order value (rounded to 2 decimals) for Gold members?

In [32]:
# Filter orders placed by Gold members
gold_orders = final_df[final_df["membership"] == "Gold"]

# Calculate average order value
avg_order_value_gold = gold_orders["total_amount"].mean()

# Round to 2 decimal places
avg_order_value_gold_rounded = round(avg_order_value_gold, 2)

avg_order_value_gold_rounded


np.float64(797.15)

**Answer:** The average order value for **Gold members** is **"797.15"**.


## Numerical type Question 5
**Question:** How many orders were placed for restaurants with rating ≥ 4.5?



In [34]:
# Filter orders where restaurant rating is greater than or equal to 4.5
high_rating_orders = final_df[final_df["rating"] >= 4.5]

# Count total number of such orders
count_high_rating_orders = high_rating_orders.shape[0]

count_high_rating_orders


3374

**Answer:** The total number of orders placed for restaurants with a rating **≥ 4.5** is **"3374"**.


## Numerical type Question 6
**Question:** How many orders were placed in the top revenue city among Gold members only?

In [35]:
# Filtering only Gold member orders
gold_orders_df = final_df[final_df["membership"] == "Gold"]

# Finding total revenue by city for Gold members
gold_city_revenue = (
    gold_orders_df
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

# Get the top revenue city
top_city = gold_city_revenue.index[0]

# Count number of orders in that city (Gold members only)
orders_in_top_city = gold_orders_df[gold_orders_df["city"] == top_city].shape[0]

top_city, orders_in_top_city


('Chennai', 1337)

**Answer:**  **"1337"** orders were placed in the top revenue city among Gold members only.

##Fill in the blanks question

**Question:** The total number of rows in the final merged dataset is

In [36]:
# Total number of rows in the final merged dataset
total_rows = final_df.shape[0]

total_rows


10000

The total number of rows in the final merged dataset is **"10000"**.
